# 2. Generate Converation

In [1]:
SYSTEM_PROMPT = """
You are a world-class podcast producer tasked with transforming the provided input text into an engaging and informative podcast script. The input may be unstructured or messy, sourced from PDFs or web pages. Your goal is to extract the most interesting and insightful content for a compelling podcast discussion.

# Steps to Follow:

1. **Analyze the Input:**
   Carefully examine the text, identifying key topics, points, and interesting facts or anecdotes that could drive an engaging podcast conversation. Disregard irrelevant information or formatting issues.

2. **Brainstorm Ideas:**
   In the `<scratchpad>`, creatively brainstorm ways to present the key points engagingly. Consider:
   - Analogies, storytelling techniques, or hypothetical scenarios to make content relatable
   - Ways to make complex topics accessible to a general audience
   - Thought-provoking questions to explore during the podcast
   - Creative approaches to fill any gaps in the information

3. **Craft the Dialogue:**
   Develop a natural, conversational flow between the Host (Alice) and the guest speaker (the author or an expert on the topic). Incorporate:
   - The best ideas from your brainstorming session
   - Clear explanations of complex topics
   - An engaging and lively tone to captivate listeners
   - A balance of information and entertainment

   Rules for the dialogue:
   - The Host (Alice) always initiates the conversation and interviews the guest
   - Include thoughtful questions from the host to guide the discussion
   - Incorporate natural speech patterns, including occasional verbal fillers (e.g., "um," "well," "you know")
   - Allow for natural interruptions and back-and-forth between host and guest
   - Ensure the guest's responses are substantiated by the input text, avoiding unsupported claims
   - Maintain a PG-rated conversation appropriate for all audiences
   - Avoid any marketing or self-promotional content from the guest
   - The host concludes the conversation

4. **Summarize Key Insights:**
   Naturally weave a summary of key points into the closing part of the dialogue. This should feel like a casual conversation rather than a formal recap, reinforcing the main takeaways before signing off.

5. **Maintain Authenticity:**
   Throughout the script, strive for authenticity in the conversation. Include:
   - Moments of genuine curiosity or surprise from the host
   - Instances where the guest might briefly struggle to articulate a complex idea
   - Light-hearted moments or humor when appropriate
   - Brief personal anecdotes or examples that relate to the topic (within the bounds of the input text)

6. **Consider Pacing and Structure:**
   Ensure the dialogue has a natural ebb and flow:
   - Start with a strong hook to grab the listener's attention
   - Gradually build complexity as the conversation progresses
   - Include brief "breather" moments for listeners to absorb complex information
   - End on a high note, perhaps with a thought-provoking question or a call-to-action for listeners

IMPORTANT RULE: Each line of dialogue should be no more than 100 characters (e.g., can finish within 5-8 seconds)

Remember: Always reply in valid JSON format, without code blocks. Begin directly with the JSON output.
"""

In [2]:
from typing import Literal, List

from pydantic import BaseModel, Field


class DialogueItem(BaseModel):
    """A single dialogue item."""

    speaker: Literal["Host (Alice)", "Guest"]
    text: str

class MediumDialogue(BaseModel):
    """The dialogue between the host and guest."""

    scratchpad: str
    name_of_guest: str
    dialogue: List[DialogueItem] = Field(
        ..., description="A list of dialogue items, typically between 29 to 39 items"
    )

In [3]:
with open("KFE_paper_sample_pdfreader.txt", "r", encoding="utf-8") as f:
    input_text = f.read()

In [ ]:
from utils import call_llm

modified_system_prompt = SYSTEM_PROMPT
modified_system_prompt += "\n\nAim for a moderate length, about 3-5 minutes."
modified_system_prompt += "\n\nOUTPUT LANGUAGE <IMPORTANT>: The podcast should be English."

# Call the LLM for the first time
first_draft_dialogue = call_llm(modified_system_prompt, input_text, MediumDialogue)

In [5]:
first_draft_dialogue.to_dict()

{'id': 'chatcmpl-Bpc8EKI8HXKbfjWJfrZuJfVEYlPTI',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"scratchpad":"Key topics: Radioactivation in fusion reactors, decay heat, cooling strategies, waste classification, K-DEMO specs. Hook: The hidden challenge of fusion—managing radioactive leftovers. Use analogies (like cooling down a hot oven), explain decay heat simply, and highlight the balance between safety and efficiency. Questions: Why is decay heat a problem? How do you keep components safe? What happens to the waste? Use a conversational tone, with Jane expressing surprise at the high temperatures and the need for careful cooling. End with a reflection on the future of fusion safety.","name_of_guest":"Dr. Beom Seok Kim","dialogue":[{"speaker":"Host (Alice)","text":"Welcome to Science Unplugged! Today, we\'re diving into fusion reactors."},{"speaker":"Host (Alice)","text":"I\'m joined by Dr. Beom Seok Kim from the National Fusion

In [6]:
# Call the LLM a second time to improve the dialogue
system_prompt_with_dialogue = f"{modified_system_prompt}\n\nHere is the first draft of the dialogue you provided:\n\n{first_draft_dialogue.model_dump_json()}."
final_dialogue = call_llm(system_prompt_with_dialogue, "Please improve the dialogue. Make it more natural and engaging.", MediumDialogue)

In [7]:
final_dialogue.to_dict()

{'id': 'chatcmpl-Bpc8Q3a78GYHo1tXkRxjmc1mpBqPw',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"scratchpad":"Key topics: Radioactivation in fusion reactors, decay heat, cooling strategies, waste classification, K-DEMO specs. Hook: The hidden challenge of fusion—managing radioactive leftovers. Use analogies (like cooling down a hot oven), explain decay heat simply, and highlight the balance between safety and efficiency. Questions: Why is decay heat a problem? How do you keep components safe? What happens to the waste? Use a conversational tone, with Jane expressing surprise at the high temperatures and the need for careful cooling. Add more natural back-and-forth, light humor, and relatable examples. End with a hopeful look at fusion\'s future.","name_of_guest":"Dr. Beom Seok Kim","dialogue":[{"speaker":"Host (Alice)","text":"Hey everyone, welcome back to Science Unplugged! I\'m Jane, your host."},{"speaker":"Host (Alice)","text"

In [8]:
import json
result = json.loads(final_dialogue.choices[0].message.content)
result["dialogue"]

[{'speaker': 'Host (Alice)',
  'text': "Hey everyone, welcome back to Science Unplugged! I'm Jane, your host."},
 {'speaker': 'Host (Alice)',
  'text': "Today, we're cracking open the world of fusion reactors—no lab coat required!"},
 {'speaker': 'Host (Alice)',
  'text': 'Joining me is Dr. Beom Seok Kim from the National Fusion Research Institute.'},
 {'speaker': 'Host (Alice)',
  'text': 'Dr. Kim, thanks for coming on. Ready for some tough questions?'},
 {'speaker': 'Guest',
  'text': 'Absolutely, Jane. Fire away—hopefully not literally!'},
 {'speaker': 'Host (Alice)',
  'text': "Ha! Okay, fusion is called 'clean' nuclear energy. But is it really waste-free?"},
 {'speaker': 'Guest',
  'text': 'Great question. Fusion avoids the nastiest waste, but it’s not totally spotless.'},
 {'speaker': 'Host (Alice)',
  'text': 'So, what actually gets radioactive in a fusion reactor?'},
 {'speaker': 'Guest',
  'text': 'Mainly the parts inside the reactor—like blankets and divertors. Neutrons hit t

In [9]:
with open("KFE_paper_sample_conversation.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(result["dialogue"], indent=4, ensure_ascii=False))